# Chapter 3 スクレイピングでデータを収集しよう

## Lesson 21 Webページを取得してみましょう

### 手順パート: 書籍ページのHTMLを取得する

『いちばんやさしいPythonの教本』の書籍ページを取得する

In [2]:
import requests  # requestsをインポート
res = requests.get('https://book.impress.co.jp/books/1116101151')  # 書籍ページのURLを指定して、レスポンスを取得
res.status_code  # ステータスコードを確認

200

In [3]:
html_doc = res.text
print(html_doc[:150])





<!DOCTYPE html>
<html lang="ja" dir="ltr">
<head>
<meta charset="utf-8" />

<title>いちばんやさしいPythonの教本 人気講師が教える基礎からサーバサイド開発まで - インプレスブックス</title>

<


## Lesson 22 Webページをスクレイピングしましょう

In [4]:
from bs4 import BeautifulSoup

html_doc = '''
<html>
    <head>
        <title>いちやさサンプル</title>
    </head>
    <body>
        <h1 class="not_link">スクレイピング用サンプル</h1>
        <ul id="first_ul">
            <li><a href="http://ichiyasa.sample/link1">リンク1</a></li>
            <li><a href="http://ichiyasa.sample/link2">リンク2</a></li>
            <li class="not_link">テキスト1</li>
        </ul>
        <ul id="second_ul">
            <li><a href="http://ichiyasa.sample/link3">リンク3</a></li>
            <li class="not_link">テキスト2</li>
        </ul>
    </body>
</html>
'''

soup = BeautifulSoup(html_doc, 'html.parser')

In [5]:
a_tag = soup.find('a')  # 最初のa要素を検索
print('内容:', a_tag.get_text())  # 要素の内容を取得
print('href属性:', a_tag['href'])  # 要素の属性値を取得

内容: リンク1
href属性: http://ichiyasa.sample/link1


In [6]:
# id属性で検索
soup.find(id='second_ul')

<ul id="second_ul">
<li><a href="http://ichiyasa.sample/link3">リンク3</a></li>
<li class="not_link">テキスト2</li>
</ul>

In [7]:
# class属性で検索
soup.find(class_='not_link')

<h1 class="not_link">スクレイピング用サンプル</h1>

In [8]:
# 要素名とclass属性で検索
soup.find('li', class_='not_link')

<li class="not_link">テキスト1</li>

In [9]:
# テキスト2のli要素を検索
ul_tag = soup.find('ul', id='second_ul')
ul_tag.find('li', class_='not_link')

<li class="not_link">テキスト2</li>

In [10]:
# 全てのa要素を取得
a_tags = soup.find_all('a')
for a_tag in a_tags:
    print(a_tag)

<a href="http://ichiyasa.sample/link1">リンク1</a>
<a href="http://ichiyasa.sample/link2">リンク2</a>
<a href="http://ichiyasa.sample/link3">リンク3</a>


In [11]:
# 複数の要素名で検索
multi_tags = soup.find_all(['h1', 'a'])
for tag in multi_tags:
    print(tag)

<h1 class="not_link">スクレイピング用サンプル</h1>
<a href="http://ichiyasa.sample/link1">リンク1</a>
<a href="http://ichiyasa.sample/link2">リンク2</a>
<a href="http://ichiyasa.sample/link3">リンク3</a>


### 手順パート: 書籍ページから書籍名と値段を取得する

In [12]:
import requests
from bs4 import BeautifulSoup

res = requests.get('https://book.impress.co.jp/books/1116101151')
html_doc = res.text
soup = BeautifulSoup(html_doc, 'html.parser')

In [13]:
div_book_detail = soup.find('div', class_='block-book-detail')
div_book_detail

<div class="block-book-detail">
<div class="block-book-detail-head">
<h2>いちばんやさしいPythonの教本 人気講師が教える基礎からサーバサイド開発まで</h2>
</div>
<div class="block-book-detail-img">
<div class="module-book-img">
<p><img alt="いちばんやさしいPythonの教本 人気講師が教える基礎からサーバサイド開発まで" src="//img.ips.co.jp/ij/16/1116101151/1116101151-520x.jpg" width="260"/></p>
</div>
<div class="module-book-info">
<p class="module-book-price">¥2,200＋税</p>
<dl class="module-book-data">
<dt>品種名</dt><dd>書籍</dd>
<dt>発売日</dt><dd>2017/8/10</dd>
<dt>ページ数</dt><dd>272</dd>
<dt>サイズ</dt><dd>B5変形判</dd>
<dt>著者</dt><dd>
鈴木 たかのり　著/杉谷 弥月　著/株式会社ビープラウド　著
</dd>
<dt>ISBN</dt><dd>
9784295002086</dd>
</dl>
</div>
<!--//block-book-detail-img--></div>
<div class="block-book-detail-body">
<div class="module-book-link-tab">
<ul class="module-inline-block"><li class="module-otameshi"><a href="http://impress.tameshiyo.me/9784295002086" target="_blank">試し読み</a></li><li class="module-download"><a href="#box-download">ダウンロード</a></li></ul>
</div>
<div class="module-book-d

In [14]:
book_title = div_book_detail.find('h2')
book_title.get_text()

'いちばんやさしいPythonの教本 人気講師が教える基礎からサーバサイド開発まで'

In [15]:
book_price = div_book_detail.find('p', class_='module-book-price')
book_price.get_text()

'¥2,200＋税'

## Lesson 23 少し難しいスクレイピングに挑戦しましょう

In [16]:
html_doc = '''
<dl id='python-books'>
  <dt>Python入門本</dt><dd>1000円</dd>
  <dt>スクレイピング本</dt><dd>1500円</dd>
  <dt>機械学習本</dt><dd>2000円</dd>
</dl>
'''

In [17]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(html_doc, 'html.parser')
python_books = {}
for tag in soup.find_all(['dt', 'dd']):
    if tag.name == 'dt':  # dt要素の場合はキー
        key = tag.get_text()
    if tag.name  == 'dd':  # dd要素の場合は値
        python_books[key] = tag.get_text()

print('Python本の辞書: ', python_books)
print('スクレイピング本の値段: ', python_books['スクレイピング本'])

Python本の辞書:  {'Python入門本': '1000円', 'スクレイピング本': '1500円', '機械学習本': '2000円'}
スクレイピング本の値段:  1500円


### 手順パート: 書籍ページから発売日と著者を取得する

In [18]:
import requests
from bs4 import BeautifulSoup

res = requests.get('https://book.impress.co.jp/books/1116101151')
html_doc = res.text
soup = BeautifulSoup(html_doc, 'html.parser')
div_book_detail = soup.find('div', class_='block-book-detail')

In [19]:
dl_book_data = div_book_detail.find('dl', class_='module-book-data')
dl_book_data

<dl class="module-book-data">
<dt>品種名</dt><dd>書籍</dd>
<dt>発売日</dt><dd>2017/8/10</dd>
<dt>ページ数</dt><dd>272</dd>
<dt>サイズ</dt><dd>B5変形判</dd>
<dt>著者</dt><dd>
鈴木 たかのり　著/杉谷 弥月　著/株式会社ビープラウド　著
</dd>
<dt>ISBN</dt><dd>
9784295002086</dd>
</dl>

In [20]:
book_data = {}
for tag in dl_book_data.find_all(['dt', 'dd']):
    if tag.name == 'dt':
        key = tag.get_text()
    if tag.name  == 'dd':
        book_data[key] = tag.get_text().strip()
        
book_data

{'品種名': '書籍',
 '発売日': '2017/8/10',
 'ページ数': '272',
 'サイズ': 'B5変形判',
 '著者': '鈴木 たかのり\u3000著/杉谷 弥月\u3000著/株式会社ビープラウド\u3000著',
 'ISBN': '9784295002086'}

In [21]:
print('発売日:', book_data['発売日'])
print('著者:', book_data['著者'])

発売日: 2017/8/10
著者: 鈴木 たかのり　著/杉谷 弥月　著/株式会社ビープラウド　著


## Lesson 24 複数のWebページからデータを集めましょう

### 手順パート: 複数の書籍ページをスクレイピングする

In [22]:
import requests
from bs4 import BeautifulSoup

res = requests.get('https://book.impress.co.jp/booklist/')
html_doc = res.text
soup = BeautifulSoup(html_doc, 'html.parser')
div_book_list = soup.find('div', class_='block-book-list-body')

In [23]:
book_urls = []  # 書籍ページのURLリスト
a_tags = div_book_list.find_all('a')
for a_tag in a_tags:
    if a_tag['href'] not in book_urls:  # 重複しないように存在チェック
        book_urls.append(a_tag['href'])
book_urls

['https://book.impress.co.jp/books/1118101062',
 'https://book.impress.co.jp/books/1118101144',
 'https://book.impress.co.jp/books/1118101132',
 'https://book.impress.co.jp/books/1118101106',
 'https://book.impress.co.jp/books/1118170106',
 'https://book.impress.co.jp/books/1118501019',
 'https://book.impress.co.jp/books/1118110115',
 'https://book.impress.co.jp/books/1118102041',
 'https://book.impress.co.jp/books/1118101103',
 'https://book.impress.co.jp/books/1118101130',
 'https://book.impress.co.jp/books/1118170129',
 'https://book.impress.co.jp/books/1117101058',
 'https://book.impress.co.jp/books/1117170066',
 'https://book.impress.co.jp/books/1118101099',
 'https://book.impress.co.jp/books/1118101085',
 'https://book.impress.co.jp/books/1118101151',
 'https://book.impress.co.jp/books/1118170151',
 'https://book.impress.co.jp/books/1118101149',
 'https://book.impress.co.jp/books/1118170149',
 'https://book.impress.co.jp/books/1118101150',
 'https://book.impress.co.jp/books/11181

In [24]:
def get_book_info(book_url):
    # 書籍ページをスクレイピングする準備
    res = requests.get(book_url)
    html_doc = res.text
    soup = BeautifulSoup(html_doc, 'html.parser')
    
    # 書籍情報のブロック
    div_book_detail = soup.find('div', class_='block-book-detail')

    # 書籍名
    book_title = div_book_detail.find('h2')
    # 値段
    book_price = div_book_detail.find('p', class_='module-book-price')
    
    # 発売日・著者
    book_data = {}
    dl_book_data = div_book_detail.find('dl', class_='module-book-data')
    for tag in dl_book_data.find_all(['dt', 'dd']):
        if tag.name == 'dt':
            key = tag.get_text()
        if tag.name  == 'dd':
            book_data[key] = tag.get_text().strip()

    return [
        book_title.get_text(),  # 書籍名
        book_price.get_text(),  # 値段
        book_data['発売日'],  # 発売日
        book_data['著者'],  # 著者
    ]

In [25]:
get_book_info('https://book.impress.co.jp/books/1116101151')

['いちばんやさしいPythonの教本 人気講師が教える基礎からサーバサイド開発まで',
 '¥2,200＋税',
 '2017/8/10',
 '鈴木 たかのり\u3000著/杉谷 弥月\u3000著/株式会社ビープラウド\u3000著']

In [26]:
import time

book_info_list = []
for book_url in book_urls:
    print('スクレイピング中: ', book_url)
    book_info_list.append(get_book_info(book_url))
    time.sleep(1)
print('完了')

スクレイピング中:  https://book.impress.co.jp/books/1118101062
スクレイピング中:  https://book.impress.co.jp/books/1118101144
スクレイピング中:  https://book.impress.co.jp/books/1118101132
スクレイピング中:  https://book.impress.co.jp/books/1118101106
スクレイピング中:  https://book.impress.co.jp/books/1118170106
スクレイピング中:  https://book.impress.co.jp/books/1118501019
スクレイピング中:  https://book.impress.co.jp/books/1118110115
スクレイピング中:  https://book.impress.co.jp/books/1118102041
スクレイピング中:  https://book.impress.co.jp/books/1118101103
スクレイピング中:  https://book.impress.co.jp/books/1118101130
スクレイピング中:  https://book.impress.co.jp/books/1118170129
スクレイピング中:  https://book.impress.co.jp/books/1117101058
スクレイピング中:  https://book.impress.co.jp/books/1117170066
スクレイピング中:  https://book.impress.co.jp/books/1118101099
スクレイピング中:  https://book.impress.co.jp/books/1118101085
スクレイピング中:  https://book.impress.co.jp/books/1118101151
スクレイピング中:  https://book.impress.co.jp/books/1118170151
スクレイピング中:  https://book.impress.co.jp/books/1118101149
スクレイピング中: 

In [27]:
book_info_list

[['いちばんやさしいJavaScriptの教本 第2版 ECMAScript 2017(ES8)対応 人気講師が教えるWebプログラミング入門',
  '¥2,200＋税',
  '2019/3/22',
  '岩田 宇史\u3000著'],
 ['スラスラ読める Rubyふりがなプログラミング',
  '¥2,000＋税',
  '2019/3/15',
  'リブロワークス\u3000著/高橋 征義\u3000監修'],
 ['できるExcel関数 Office 365/2019/2016/2013/2010対応 データ処理の効率アップに役立つ本',
  '¥1,580＋税',
  '2019/3/14',
  '尾崎 裕子\u3000著/できるシリーズ編集部\u3000著'],
 ['年間70億円投資！行列ができる建築会社が実践する 最強ホテルプロジェクト',
  '¥1,600＋税',
  '2019/3/25',
  '佐々木 数修都\u3000著'],
 ['できるポケット 時短の王道 Excel関数全事典 改訂版 Office 365 & Excel 2019/2016/2013/2010対応',
  '¥1,380＋税',
  '2019/3/22',
  '羽山 博\u3000著/吉川 明広\u3000著/できるシリーズ編集部\u3000著'],
 ['おしゃれな季節とイベントのお知らせ＆チラシ素材集', '¥1,980＋税', '2019/3/14', 'Power Design\u3000他著'],
 ['ハンドメイド作家のための教科書 minneが教える売れるきほん帖',
  '¥1,600＋税',
  '2019/4/5',
  'minne作家活動アドバイザー 和田まお\u3000著'],
 ['徹底攻略 ネットワークスペシャリスト教科書 2019年度',
  '¥2,780＋税',
  '2019/3/15',
  '株式会社わくわくスタディワールド 瀬戸美月\u3000著'],
 ['できるポケット Windows 10 基本＆活用マスターブック 改訂4版',
  '¥800＋税',
  '2019/3/25',
  '一ヶ谷兼乃\u3000著/清水理史\u3000著/できるシリーズ編集部\u3000著/法林岳之\u3000著'],

### 手順パート: 収集した書籍情報を保存する

In [28]:
with open('book_data.tsv', 'w', encoding='utf-8') as f:
    for book_info in book_info_list:
        f.write('\t'.join(book_info) + '\n')

### 手順パート: TSVファイルから書籍を検索する

In [29]:
def book_search(keyword):  # キーワードを受け取る
    results = []  # キーワードが含まれる行を格納する
    with open('book_data.tsv', encoding='utf-8') as f:  # tsvファイルを開く
        for line in f:  # 各行を読み込む
            cols = line.split('\t')  # タブ文字で分割する
            if keyword in cols[0]:  # 書籍名にキーワードが含まれるか
                results.append(line)
    if len(results) > 0:  # 1件以上検索ヒットした場合
        response =  ''.join(results)  # 連結する
    else:  # 検索ヒットしたものがなかった場合
        response = '「{}」ガ含マレル書籍ガ見ツカリマセンデシタ'.format(keyword)
    return response

In [30]:
response = book_search('いちばんやさしい')
print(response)

いちばんやさしいJavaScriptの教本 第2版 ECMAScript 2017(ES8)対応 人気講師が教えるWebプログラミング入門	¥2,200＋税	2019/3/22	岩田 宇史　著
いちばんやさしい資料作成＆プレゼンの教本 人気講師が教える「人の心つかむプレゼン」のすべて	¥1,980＋税	2019/4/5	髙橋惠一郎　著



In [31]:
response = book_search('寿司')
print(response)

「寿司」ガ含マレル書籍ガ見ツカリマセンデシタ
